**#0 Importy**

In [118]:
import pandas as pd
import datetime as dt
import os
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

**#1 Wczytanie baz**

In [119]:
class db:
    def __init__(self):
        self.transactions = db.transaction_init()  # Wywołanie metody statycznej
        self.cc = pd.read_csv(r'db\country_codes.csv', index_col=0)
        self.customers = pd.read_csv(r'db\customers.csv', index_col=0)
        self.prod_info = pd.read_csv(r'db\prod_cat_info.csv')

#Konwersje dat

    @staticmethod
    def transaction_init():
        transactions_list = []  # Lista do przechowywania DataFrame'ów
        src = r'db\transactions'
        
        for filename in os.listdir(src):
            transactions_list.append(pd.read_csv(os.path.join(src, filename), index_col=0))
        
        # Połącz wszystkie DataFrame'y w jeden
        transactions = pd.concat(transactions_list, ignore_index=True)
    
        def convert_dates(x):
            try:
                return dt.datetime.strptime(x, '%d-%m-%Y')
            except:
                return dt.datetime.strptime(x, '%d/%m/%Y')
    
        transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))
    
        return transactions


#łączenie kilku baz

    def merge(self):
        df = self.transactions.join(
            self.prod_info.drop_duplicates(subset=['prod_cat_code'])
            .set_index('prod_cat_code')['prod_cat'],
            on='prod_cat_code',
            how='left'
        )

        df = df.join(
            self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])
            .set_index('prod_sub_cat_code')['prod_subcat'],
            on='prod_subcat_code',
            how='left'
        )

        df = df.join(
            self.customers.join(self.cc, on='country_code')
            .set_index('customer_Id'),
            on='cust_id'
        )

        self.merged = df

# Tworzenie obiektu i wywołanie funkcji merge
df = db()
df.merge()


**#2 Budowa podstawowego layoutu**

In [120]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

In [121]:
app.layout = html.Div([
    html.Div([
        dcc.Tabs(
            id='tabs',
            value='tab-1',
            children=[
                dcc.Tab(label='Sprzedaż globalna', value='tab-1'),
                dcc.Tab(label='Produkty', value='tab-2'),
                dcc.Tab(label='Kanały sprzedaży', value='tab-3')
            ]
        ),
        html.Div(id='tabs-content')
    ], style={'width': '80%', 'margin': 'auto'})
], style={'height': '100%'})


**#3 Sprzedaż globalna**

In [122]:
def render_tab1(df):
    layout = html.Div([
        html.H1('Sprzedaż globalna', style={'text-align': 'center'}),
        html.Div([dcc.DatePickerRange(
            id='sales-range',
            start_date=df['tran_date'].min(),
            end_date=df['tran_date'].max(),
            display_format='YYYY-MM-DD'
        )], style={'width': '100%', 'text-align': 'center'}),
        html.Div([
            html.Div([dcc.Graph(id='bar-sales')], style={'width': '50%'}),
            html.Div([dcc.Graph(id='choropleth-sales')], style={'width': '50%'})
        ], style={'display': 'flex'})
    ])
    return layout

**#4 Produkty**

In [123]:
def render_tab2(df):
    grouped = df[df['total_amt'] > 0].groupby('prod_cat')['total_amt'].sum()
    fig = go.Figure(data=[go.Pie(labels=grouped.index, values=grouped.values)],
                    layout=go.Layout(title='Udział grup produktów w sprzedaży'))
    
    layout = html.Div([
        html.H1('Produkty', style={'text-align': 'center'}),
        html.Div([
            html.Div([dcc.Graph(id='pie-prod-cat', figure=fig)], style={'width': '50%'}),
            html.Div([
                dcc.Dropdown(
                    id='prod_dropdown',
                    options=[{'label': prod_cat, 'value': prod_cat} for prod_cat in df['prod_cat'].unique()],
                    value=df['prod_cat'].unique()[0]
                ),
                dcc.Graph(id='barh-prod-subcat')
            ], style={'width': '50%'})
        ], style={'display': 'flex'})
    ])
    return layout

**#5 Kanały sprzedaży**

In [124]:
def render_tab3(df):
    layout = html.Div([
        html.H1('Kanały sprzedaży', style={'text-align': 'center'}),
        html.Div([
            dcc.Dropdown(
                id='store-type-dropdown',
                options=[{'label': store, 'value': store} for store in df['Store_type'].unique()],
                value=df['Store_type'].unique()[0],
                placeholder='Wybierz kanał sprzedaży'
            )
        ], style={'width': '50%', 'margin': 'auto', 'padding-bottom': '20px'}),
        html.Div([
            html.Div([
                dcc.Graph(id='weekday-sales')  # Wykres dla dni tygodnia
            ], style={'width': '50%'}),
            html.Div([
                dcc.Graph(id='customer-demographics')  # Wykres demograficzny klientów
            ], style={'width': '50%'})
        ], style={'display': 'flex'})
    ])
    return layout

**#6 Callbacks**

In [125]:
# Callback dla zawartości zakładek
@app.callback(Output('tabs-content', 'children'), [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return render_tab1(df.merged)
    elif tab == 'tab-2':
        return render_tab2(df.merged)
    elif tab == 'tab-3':  
        return render_tab3(df.merged)

# Callbacki dla tab-1
@app.callback(Output('bar-sales', 'figure'),
              [Input('sales-range', 'start_date'), Input('sales-range', 'end_date')])
def tab1_bar_sales(start_date, end_date):
    truncated = df.merged[(df.merged['tran_date'] >= start_date) & (df.merged['tran_date'] <= end_date)]
    grouped = truncated[truncated['total_amt'] > 0].groupby(
        [pd.Grouper(key='tran_date', freq='M'), 'Store_type']
    )['total_amt'].sum().round(2).unstack()

    traces = []
    for col in grouped.columns:
        traces.append(go.Bar(
            x=grouped.index, y=grouped[col], name=col, hoverinfo='text',
            hovertext=[f'{y/1e3:.2f}k' for y in grouped[col].values]
        ))

    fig = go.Figure(data=traces, layout=go.Layout(title='Przychody', barmode='stack', legend=dict(x=0, y=-0.5)))
    return fig

@app.callback(Output('choropleth-sales', 'figure'),
              [Input('sales-range', 'start_date'), Input('sales-range', 'end_date')])
def tab1_choropleth_sales(start_date, end_date):
    truncated = df.merged[(df.merged['tran_date'] >= start_date) & (df.merged['tran_date'] <= end_date)]
    grouped = truncated[truncated['total_amt'] > 0].groupby('country')['total_amt'].sum().round(2)

    trace0 = go.Choropleth(
        colorscale='Viridis', reversescale=True,
        locations=grouped.index, locationmode='country names',
        z=grouped.values, colorbar=dict(title='Sales')
    )
    fig = go.Figure(data=[trace0], layout=go.Layout(title='Mapa', geo=dict(showframe=False, projection={'type': 'natural earth'})))
    return fig

# Callback dla tab-2
@app.callback(Output('barh-prod-subcat', 'figure'),
              [Input('prod_dropdown', 'value')])
def tab2_barh_prod_subcat(chosen_cat):
    grouped = df.merged[(df.merged['total_amt'] > 0) & (df.merged['prod_cat'] == chosen_cat)].pivot_table(
        index='prod_subcat', columns='Gender', values='total_amt', aggfunc='sum'
    ).assign(_sum=lambda x: x['F'] + x['M']).sort_values(by='_sum').round(2)

    traces = []
    for col in ['F', 'M']:
        traces.append(go.Bar(x=grouped[col], y=grouped.index, orientation='h', name=col))

    fig = go.Figure(data=traces, layout=go.Layout(barmode='stack', margin={'t': 20}))
    return fig

# Callback dla tab-3
@app.callback(Output('weekday-sales', 'figure'),
              [Input('store-type-dropdown', 'value')])
def weekday_sales(store_type):
    filtered = df.merged[df.merged['Store_type'] == store_type]
    filtered['day_of_week'] = filtered['tran_date'].dt.day_name()

    # Tworzenie słownika, który przypisuje dniom tygodnia odpowiednią wartość numeryczną
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    filtered['day_of_week_order'] = filtered['day_of_week'].apply(lambda x: day_order.index(x))

    # Grupowanie po dniu tygodnia i sumowanie sprzedaży
    grouped = filtered.groupby('day_of_week')['total_amt'].sum()

    # Sortowanie dni tygodnia zgodnie z kolejnością
    grouped = grouped.loc[day_order]

    fig = go.Figure(data=[go.Bar(x=grouped.index, y=grouped.values)],
                    layout=go.Layout(title=f'Sprzedaż w dniach tygodnia ({store_type})',
                                     xaxis=dict(title='Dzień tygodnia'),
                                     yaxis=dict(title='Sprzedaż')))
    return fig
@app.callback(Output('customer-demographics', 'figure'),
              [Input('store-type-dropdown', 'value')])
def customer_demographics(store_type):
    filtered = df.merged[df.merged['Store_type'] == store_type]
    gender_counts = filtered['Gender'].value_counts()

    fig = go.Figure(data=[go.Pie(labels=gender_counts.index, values=gender_counts.values)],
                    layout=go.Layout(title=f'Udział płci dla {store_type}'))
    return fig

In [126]:
if __name__ == '__main__':
    app.run_server(debug=True)

C:\Users\Mateusz\AppData\Local\Temp\ipykernel_15384\1422908395.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\Mateusz\AppData\Local\Temp\ipykernel_15384\1422908395.py:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Mateusz\AppData\Local\Temp\ipykernel_15384\1422908395.py:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Mateusz\AppData\Local\Temp\ipykernel_15384\1422908395.py:64: SettingWithCopyWarning:


A value is tr